In [ ]:
from ratdata import data_manager as dm
from pathlib import Path
import re
from datetime import datetime

In [ ]:
gui_dirname = 'data/gui_recordings'

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
for f in Path(gui_dirname).iterdir():
    if not re.match(r'^.*\.txt$', f.name):
        continue
    rat_shortlabel = re.findall(r'rat[0-9]', f.stem)[0]
    date = re.findall(r'^[0-9]{4}-[0-9]{2}-[0-9]{2}', f.stem)[0]
    time = re.findall(r'[0-9]{2}-[0-9]{2}-[0-9]{2}', f.stem)[1]
    rat = dm.Rat.select().where((dm.Rat.label == rat_shortlabel) & (dm.Rat.date_start <= date) & (dm.Rat.date_end >= date)).get()
    recordings = dm.RecordingFile.select().join(dm.StimSettings).where((dm.RecordingFile.rat == rat) & (dm.RecordingFile.recording_date == date) & (dm.StimSettings.stim_type == 'proportional'))
    selected_recording = None
    min_diff = None
    for r in recordings:
        datetime_string_recording = r.filename.split(' ')[0]
        datetime_string_gui = 'T'.join([date, time])
        
        rec_time = datetime.strptime(datetime_string_recording, '%Y-%m-%dT%H-%M-%S')
        gui_time = datetime.strptime(datetime_string_gui, '%Y-%m-%dT%H-%M-%S')
        time_diff = abs((gui_time - rec_time).total_seconds())
        if selected_recording is None:
            selected_recording = r
            min_diff = time_diff
        elif time_diff < min_diff:
            selected_recording = r
            min_diff = time_diff
    if dm.GuiAmplitude.select().where(dm.GuiAmplitude.file == f.name).count() == 0:
        print(f.name, rat.full_label, selected_recording.filename, min_diff)
        dm.GuiAmplitude.insert(recording=selected_recording, offset=0, file=f.name).execute()